In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!pip install xgboost

##Load the preprocessed Dataset

In [ ]:
import numpy as np       
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import xgboost as xgb
import pickle
from geopy.geocoders import Nominatim
from sklearn.model_selection import train_test_split

path = "/content/drive/MyDrive/Bachelorarbeit/PreProcessed_NYC_GreenTaxi.csv"
df = pd.read_csv(path)

In [ ]:
df.describe()

Identify which features are of object type, change them to float or integer and finally add few features to train our model.

In [ ]:
df.info()

In [ ]:
df["dropoff_datetime"] = pd.to_datetime(df["dropoff_datetime"], format='%Y-%m-%d %H:%M:%S')
df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"], format='%Y-%m-%d %H:%M:%S')
df["pickup_weekday"] = df["pickup_datetime"].dt.weekday
df["dropoff_weekday"] = df["dropoff_datetime"].dt.weekday
# Get latitude and longitude differences
df["latitude_difference"] = df["dropoff_latitude"] - df["pickup_latitude"]
df["longitude_difference"] = df["dropoff_longitude"] - df["pickup_longitude"]

## Train and Save the XGBoost Model
#### The following features were used to train the model and predict trip duration in minutes: *'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'trip_distance', 'pickup_weekday', 'pickup_hour', 'pickup_minute'.*


In [ ]:
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
from statsmodels.graphics.api import abline_plot
import seaborn as sns


def rmsle(y_true, y_pred):
    labels = y_pred.get_label()
    diffs = np.log(y_true + 1) - np.log(labels + 1)
    squared_diffs = np.square(diffs)
    avg = np.mean(squared_diffs)
    return ('RMSLE', np.sqrt(avg))

def XGBmodel(X, y):

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
  dtrain = xgb.DMatrix(X_train, np.log(y_train+1))
  dtest = xgb.DMatrix(X_test, np.log(y_test+1))

  params = {
    'min_child_weight': 1, 
    'learning_rate': 0.1, 
    'colsample_bytree': 0.7, 
    'max_depth': 14,
    'subsample': 0.9,
    'n_estimators': 500,
    'n_jobs': -1, 
    'booster' : 'gbtree', 
    'silent': 1,
    'feval': 'rmsle',
    }
  watchlist = [(dtest, 'eval'), (dtrain, 'train')]
  # Number of training rounds
  nrounds = 500
  # Train model
  gbm = xgb.train(params, dtrain, num_boost_round = nrounds, evals = watchlist, feval=rmsle,  verbose_eval = True)

  # Test predictions
  y_pred = np.exp(gbm.predict(xgb.DMatrix(X_test))) - 1
  # Mean absolute error to get a basic estimate of the error
  mae = (abs(y_pred - y_test)).mean()
  print('Mean Absolute Error:', mae)
  print(max(y_pred))

  # plotting residuals
  residuals = y_test - y_pred
  max_error = max(residuals) if abs(max(residuals)) > abs(min(residuals)) else min(residuals)
  print("Max Error:", "{:,.0f}".format(max_error))
  

  #Frequency Distributions of Residuals
  plt.figure(figsize=(10,7))
  plt.hist(residuals, bins=60, edgecolor='black')
  plt.xlabel('Residuals', fontsize = 18)
  plt.ylabel('Count', fontsize = 18)
  plt.title('Frequency Distributions of Residuals', fontsize = 18)
  plt.tick_params(axis='both', which='major', labelsize=18)
  plt.yscale('log')
  plt.show()

  return gbm

X = df[['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'trip_distance', 'pickup_weekday', 'pickup_hour', 'pickup_minute']]
y = df['trip_duration_min']
model = XGBmodel(X, y)
filename = "nyc_xgboost_model.sav"
pickle.dump(model, open(filename, 'wb'))
X.describe()



## Load the saved model

In [ ]:
filename = "nyc_xgboost_model.sav"
loaded_model = pickle.load(open(filename, 'rb'))

## Install the scikit-opt Library for the Algorithms

In [ ]:
!pip install sko
!pip install scikit-opt

##Objective Function
##### The following code was borrowed from these repositories and modified to apply Genetic Algorithms to the problem. 
1. https://github.com/khanhnamle1994/trip-optimizer/blob/master/Bio-Inspired-Algorithms/genetic_evo_main.py#L1
2. https://github.com/guofei9987/scikit-opt/blob/master/examples/demo_ga_udf_tsp.py


In [ ]:
from sko.GA import GA_TSP
from scipy import spatial
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import pprint

geolocator = Nominatim(user_agent="aco-application")

filename = "nyc_xgboost_model.sav"
loaded_model = pickle.load(open(filename, 'rb'))

def time_cost_between_points(loc1, loc2, pickup_weekday, pickup_hour, pickup_minute):
    """
    Calculate the time (in minutes) between two points
    using the trained XGB model
    """

    model_data = {
                  'pickup_longitude': loc1['x'],
                  'pickup_latitude': loc1['y'],
                  'dropoff_longitude': loc2['x'],
                  'dropoff_latitude': loc2['y'],
                  'trip_distance': trip_distance_cost(loc1, loc2),
                  'pickup_weekday': pickup_weekday,
                  'pickup_hour': pickup_hour,
                  'pickup_minute': pickup_minute,
                  }

    print(model_data)
    df = pd.DataFrame([model_data], columns=model_data.keys())
    pred = np.exp(loaded_model.predict(xgb.DMatrix(df))) - 1
    #print(pred[0])
    return pred[0]


def trip_distance_cost(loc1, loc2):
    """
    Calculate the manhattan distance between two points using
    polar coordinates in taxicab geometry https://en.wikipedia.org/wiki/Taxicab_geometry
    """
    return 0.621371 * 6371 * (
        abs(2 * np.arctan2(np.sqrt(np.square(
            np.sin((abs(loc2['y'] - loc1['y']) * np.pi / 180) / 2))),
            np.sqrt(1-(np.square(np.sin((abs(loc2['y'] - loc1['y']) * np.pi / 180) / 2)))))) +
        abs(2 * np.arctan2(np.sqrt(np.square(np.sin((abs(loc2['x'] - loc1['x']) * np.pi / 180) / 2))),
                           np.sqrt(1-(np.square(np.sin((abs(loc2['x'] - loc1['x']) * np.pi / 180) / 2)))))))


def total_cost_from_path(path):

    total_cost = 0
    for i in range(1, len(path)):
        j = i - 1
        #total_cost += np.array(cost_matrix[path[j]][path[i]])[:,0]
        total_cost += cost_matrix[path[j]][path[i]]
    # Find time it takes to go back to original location
    total_cost += cost_matrix[path[-1]][path[0]]
    #total_cost = np.array(total_cost) ##modified here
    return total_cost
    #print('total cost', total_cost)



locations = []
points = []

for index, row in df.iloc[:99].iterrows(): #take the first 50 rows within the dataset as input coordinates

        locations.append({
              'index': index,
              'x': row['pickup_longitude'],
              'y': row['pickup_latitude']
          })
        points.append((row['pickup_longitude'], row['pickup_latitude']))

# Build complete cost matrix based on time between points
cost_matrix = []
rank = len(locations)
for i in range(rank):
    row = []
    for j in range(rank):
        row.append(time_cost_between_points(locations[i], locations[j],  3, 15, 40))
    cost_matrix.append(row)
print(cost_matrix)


# 1. Genetic Algorithms

In [ ]:
import time
num_points = 99
population = 90
generations = 1999
mutation_factor = 1

start_time = time.time()
ga_tsp = GA_TSP(func=total_cost_from_path, n_dim=num_points, size_pop=population, max_iter=generations, prob_mut=mutation_factor)
best_path, best_time_cost = ga_tsp.run()
print("Execution time: %s seconds " % (time.time() - start_time))


print('Final cost: {} minutes, path: {}, for {} generations' .format(best_time_cost, best_path, str(ga_tsp.max_iter+1)))
print("Min cost mean:", np.mean(ga_tsp.generation_best_Y))
print("Min cost standard deviation:", np.std(ga_tsp.generation_best_Y))
print("Time cost in each iteration:", (ga_tsp.generation_best_Y))
#exact route adresses
'''
print("Final path addresses:")
try:
    addresses = []
    for p in best_path:
        addresses.append(geolocator.reverse(
            f"{points[p][1]}, {points[p][0]}").address)
    pprint.pprint(addresses)
except GeocoderTimedOut as e:
    print(f"Error: geocode failed with message {e}")
'''


points_arr = np.array(points)
#fig, ax = plt.subplots(1, 2)
best_points_ = np.concatenate([best_path, [best_path[0]]])
best_points_coordinate = points_arr[best_points_, :]
'''
ax[0].plot(best_points_coordinate[:, 0], best_points_coordinate[:, 1], 'o-r')
ax[0].set_xlabel('Longitude')
ax[0].set_xlabel('Latuitude')
ax[0].set_title('Trip Coordinates')
'''
plt.plot(ga_tsp.generation_best_Y, label = "GA")
plt.xlabel('Generation')
plt.ylabel('Best Cost')
plt.title("Best Cost vs Generation for " + str(ga_tsp.max_iter+1) + " Population")
plt.show()

#print('total cost', cost_matrix[0,0])

# plot lines
plt.plot(ga_tsp.generation_best_Y, label = "GA NYC")
#plt.plot(aca.generation_best_Y, label = "ACA")
# plt.plot(sa_tsp.generation_best_Y, label = "SA")
plt.xlabel('Generation', fontsize=16)
plt.ylabel('Best Cost', fontsize=16)
plt.yticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title("Best Cost vs Generation for 100 locations", fontsize=16)

#plt.title("GA NYC Taxi Best Cost vs Generation for " + str(ga_tsp.size_pop) + " Generations")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import FormatStrFormatter

best_x_history = ga_tsp.all_history_Y

fig2, ax2 = plt.subplots(1, 1)
ax2.set_title('Trip Coordinates', loc='center')
line = ax2.plot(best_points_coordinate[:, 0], best_points_coordinate[:, 1],
                marker='o', markerfacecolor='b', color='c', linestyle='-')
ax2.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax2.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax2.set_xlabel("Longitude")
ax2.set_ylabel("Latitude")
plt.ion()
p = plt.show()


def update_scatter(frame):
    ax2.set_title('iter = ' + str(frame))
    points = best_x_history[frame]
    points = np.concatenate([points, [points[0]]])
    point_coordinate = best_points_coordinate[points, :]
    plt.setp(line, 'xdata', point_coordinate[:, 0], 'ydata', point_coordinate[:, 1])
    return line


ani = FuncAnimation(fig2, update_scatter, blit=True, interval=100, frames=len(best_x_history))
plt.show()
#ani.save('ga1.gif', writer='pillow')

# 2. Ant Colony Optimization (scikit-opt)

In [ ]:
from sko.ACA import ACA_TSP

num_points = 99
population = 40
generations = 799 #2799

start_time = time.time()
aca = ACA_TSP(func=total_cost_from_path, n_dim=num_points,
              size_pop=population, max_iter=generations,
              distance_matrix=cost_matrix, alpha=1, beta=5)
best_path, best_time_cost = aca.run()
print("Execution time: %s seconds " % (time.time() - start_time))

print('Final cost: {} minutes, path: {}, for {} generations' .format(best_time_cost, best_path, str(aca.max_iter+1)))
print("Min cost mean:", np.mean(aca.generation_best_Y))
print("Min cost standard deviation:", np.std(aca.generation_best_Y))
#print("gen best y:", aca.generation_best_Y)
#print("gen best x:", aca.generation_best_X)
#print("x_best_history:", aca.x_best_history)
#print("y_best_history:", aca.y_best_history)
#print("best x", aca.best_x)
#print("best y", aca.best_y)




points_arr = np.array(points)
#fig, ax = plt.subplots(1, 2)
best_points_ = np.concatenate([best_path, [best_path[0]]])
#best_points_coordinate = points_arr[best_points_, :]
'''
ax[0].plot(best_points_coordinate[:, 0], best_points_coordinate[:, 1], 'o-r')
ax[0].set_xlabel('Longitude')
ax[0].set_xlabel('Latuitude')
ax[0].set_title('Trip Coordinates')
'''
plt.plot(aca.generation_best_Y, label = "ACO NYC")
#plt.plot(ga_tsp.generation_best_Y, label = "ACO NYC")
plt.xlabel('Generation', fontsize=16)
plt.ylabel('Best Cost' , fontsize=16)
plt.title("Best Cost vs Generation for 50 points and 30 ants" , fontsize=16)
plt.yticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import FormatStrFormatter


best_x_history = aca.x_best_history

fig2, ax2 = plt.subplots(1, 1)
ax2.set_title('Trip Coordinates', loc='center')
line = ax2.plot(best_points_coordinate[:, 0], best_points_coordinate[:, 1],
                marker='o', markerfacecolor='b', color='c', linestyle='-')
ax2.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax2.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax2.set_xlabel("Longitude")
ax2.set_ylabel("Latitude")
plt.ion()
p = plt.show()


def update_scatter(frame):
    ax2.set_title('iter = ' + str(frame))
    points = best_x_history[frame]
    points = np.concatenate([points, [points[0]]])
    point_coordinate = best_points_coordinate[points, :]
    plt.setp(line, 'xdata', point_coordinate[:, 0], 'ydata', point_coordinate[:, 1])
    return line


ani = FuncAnimation(fig2, update_scatter, blit=True, interval=100, frames=len(best_x_history))
plt.show()
ani.save('aco.gif', writer='pillow')

# 3. Simulated Annealing (scikit-opt)

In [ ]:
from sko.SA import SA_TSP
from scipy import spatial
from matplotlib.ticker import FormatStrFormatter
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import pprint


num_points = 99

start_time = time.time()
sa_tsp = SA_TSP(func=total_cost_from_path, x0=range(num_points), T_max=1, T_min=1e-9, L=10 * num_points, max_stop_counter=500 )
best_path, best_time_cost = sa_tsp.run()
print("Execution time: %s seconds " % (time.time() - start_time))
print('Final cost: {} minutes, path: {}, for {} iterations' .format(best_time_cost, best_path, str(sa_tsp.L+1)))
print("Min cost mean:", np.mean(sa_tsp.generation_best_Y))
print("Min cost standard deviation:", np.std(sa_tsp.generation_best_Y))
print("Length:", len(sa_tsp.generation_best_Y))

# find the nth iteration -beginning of convergence-
optimal = sa_tsp.generation_best_Y
def showIndices(optimal):
    return list(filter(lambda i: optimal[i] == optimal[i+1], range(len(optimal)-2)))
indices = showIndices(optimal)
print("Starts to converge from:", indices)

#find std dev of all time cost values
individual_deviation = []
for i in range (len(optimal)):
  x = optimal[i]
  y = abs((np.mean(sa_tsp.generation_best_Y)) - x)
  individual_deviation.append(y)
print('individual deviation of all iterations', individual_deviation)


points_arr = np.array(points)
#fig, ax = plt.subplots(1, 2)
best_points_ = np.concatenate([best_path, [best_path[0]]])
best_points_coordinate = points_arr[best_points_, :]
'''
ax[0].plot(best_points_coordinate[:, 0], best_points_coordinate[:, 1], 'o-r')
ax[0].set_xlabel('Longitude')
ax[0].set_xlabel('Latuitude')
ax[0].set_title('Trip Coordinates')
'''
plt.plot(sa_tsp.generation_best_Y)
plt.ylabel('Best Cost')
plt.xlabel('Iteration')
plt.title("Iteration vs Best Cost for " + str(sa_tsp.L+1) + " Iteration")
plt.figure(figsize=(10,20))
plt.show()


In [ ]:
# plot lines
plt.plot(ga_tsp.generation_best_Y, label = "GA")
plt.plot(aca.generation_best_Y, label = "ACO")
plt.plot(sa_tsp.generation_best_Y, label = "SA")
plt.xlabel('Iterations', fontsize=16)
plt.ylabel('Best Cost', fontsize=16)
plt.yticks(fontsize=14)
plt.yticks(fontsize=14)
#plt.plot(ga_tsp.generation_best_Y, label = "GA Porto Taxi Dataset")
plt.title("Best Cost vs Iteration for 100 locations", fontsize=16)
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

best_x_history = sa_tsp.best_x_history

fig2, ax2 = plt.subplots(1, 1)
ax2.set_title('title', loc='center')
line = ax2.plot(best_points_coordinate[:, 0], best_points_coordinate[:, 1],
                marker='o', markerfacecolor='b', color='c', linestyle='-')
ax2.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax2.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax2.set_xlabel("Longitude")
ax2.set_ylabel("Latitude")
plt.ion()
p = plt.show()


def update_scatter(frame):
    ax2.set_title('iter = ' + str(frame))
    points = best_x_history[frame]
    points = np.concatenate([points, [points[0]]])
    point_coordinate = best_points_coordinate[points, :]
    plt.setp(line, 'xdata', point_coordinate[:, 0], 'ydata', point_coordinate[:, 1])
    return line


ani = FuncAnimation(fig2, update_scatter, blit=True, interval=100, frames=len(best_x_history))
plt.show()
#ani.save('sa_tsp2.gif', writer='pillow')
